## Submission Record
1. LB result of v1 - Your submission scored 0.41897

### 1. Data Import 

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import datetime
import re
import nltk
pd.set_option('display.max_columns', None)

In [2]:
#Import Training Data
train = pd.read_csv("train.csv")

In [3]:
#Import Test Data
test = pd.read_csv("test.csv")

## 2. EDA

In [4]:
#Shape
train.shape, test.shape

((404290, 6), (2345796, 3))

In [5]:
train.describe()

,id,qid1,qid2,is_duplicate
count,404290.000000,404290.000000,404290.000000,404290.000000
mean,202144.500000,217243.942418,220955.655337,0.369198
std,116708.614502,157751.700002,159903.182629,0.482588
min,0.000000,1.000000,2.000000,0.000000
25%,101072.250000,74437.500000,74727.000000,0.000000
50%,202144.500000,192182.000000,197052.000000,0.000000
75%,303216.750000,346573.500000,354692.500000,1.000000
max,404289.000000,537932.000000,537933.000000,1.000000


In [6]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


## 3. Creating some features

In [7]:
#Getting the count of words in qid1
train['qid1_count'] = train["question1"].apply(lambda x : len(str(x).split()))
train['qid2_count'] = train["question2"].apply(lambda x : len(str(x).split()))
#What is the maximum difference between the length of two 'marked' duplicate questions?
train['count12_diff']= train['qid1_count']-train['qid2_count']  
train['count12_diff'] = train['count12_diff'].abs()

In [8]:
#From Anokas
train_qs = pd.Series(train['question1'].tolist() + train['question2'].tolist()).astype(str)
test_qs = pd.Series(test['question1'].tolist() + test['question2'].tolist()).astype(str)

In [9]:
qmarks = np.mean(train_qs.apply(lambda x: '?' in x))
math = np.mean(train_qs.apply(lambda x: '[math]' in x))
fullstop = np.mean(train_qs.apply(lambda x: '.' in x))
capital_first = np.mean(train_qs.apply(lambda x: x[0].isupper()))
capitals = np.mean(train_qs.apply(lambda x: max([y.isupper() for y in x])))
numbers = np.mean(train_qs.apply(lambda x: max([y.isdigit() for y in x])))

print('Questions with question marks: {:.2f}%'.format(qmarks * 100))
print('Questions with [math] tags: {:.2f}%'.format(math * 100))
print('Questions with full stops: {:.2f}%'.format(fullstop * 100))
print('Questions with capitalised first letters: {:.2f}%'.format(capital_first * 100))
print('Questions with capital letters: {:.2f}%'.format(capitals * 100))
print('Questions with numbers: {:.2f}%'.format(numbers * 100))

Questions with question marks: 99.87%
Questions with [math] tags: 0.12%
Questions with full stops: 6.31%
Questions with capitalised first letters: 99.81%
Questions with capital letters: 99.95%
Questions with numbers: 11.83%


In [10]:
from nltk.corpus import stopwords

stops = set(stopwords.words("english"))

def word_match_share(row):
    # Creating dictionaries of each word
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))/(len(q1words) + len(q2words))
    return R

# plt.figure(figsize=(15, 5))
# train_word_match = df_train.apply(word_match_share, axis=1, raw=True)
# plt.hist(train_word_match[df_train['is_duplicate'] == 0], bins=20, normed=True, label='Not Duplicate')
# plt.hist(train_word_match[df_train['is_duplicate'] == 1], bins=20, normed=True, alpha=0.7, label='Duplicate')
# plt.legend()
# plt.title('Label distribution over word_match_share', fontsize=15)
# plt.xlabel('word_match_share', fontsize=15)


## Generating N-grams features

In [11]:
from nltk import word_tokenize, ngrams

# Functions to create n-grams
def one_gram(row):
    #Storing the data
    que1 = str(row['question1'])
    que2 = str(row['question2'])
    
    #Unigrams
    one_que1 = [word for word in que1.lower().split() if word not in stops]
    one_que2 = [word for word in que2.lower().split() if word not in stops]
    #common elements
    common_onegram = len(set(one_que1).intersection(set(one_que2)))
    #% of common stuff
    onegram_pct = float(common_onegram) / max(len(set(one_que1).union(set(one_que2))),1)
    return onegram_pct

def two_gram(row):
    
    #Storing the data
    que1 = str(row['question1'])
    que2 = str(row['question2'])
    
    #Unigrams
    one_que1 = [word for word in que1.lower().split() if word not in stops]
    one_que2 = [word for word in que2.lower().split() if word not in stops]
    #2 grams
    two_que1 = [word for word in ngrams(one_que1, 2)]
    two_que2 = [word for word in ngrams(one_que2, 2)]
    #common elements
    common_twogram = len(set(two_que1).intersection(set(two_que2)))
    #% of common stuff
    twogram_pct = float(common_twogram)/max(len(set(two_que1).union(set(two_que2))),1)
    return twogram_pct

def three_gram(row):
    #Storing the data
    que1 = str(row['question1'])
    que2 = str(row['question2'])
    
    #Unigrams
    one_que1 = [word for word in que1.lower().split() if word not in stops]
    one_que2 = [word for word in que2.lower().split() if word not in stops]
    # 3 grams
    three_que1 = [word for word in ngrams(one_que1, 3)]
    three_que2 = [word for word in ngrams(one_que2, 3)]
    #common elements
    common_threegram = len(set(three_que1).intersection(set(three_que2)))
    #% of common stuff
    threegram_pct = float(common_threegram)/max(len(set(three_que1).union(set(three_que2))),1)
    
    return threegram_pct

In [12]:
from collections import Counter
import re, math

# Function to get cosine similarity
def get_cosine(row):
    # Creating dictionaries of each word
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    vec1 = Counter(q1words)
    vec2 = Counter(q2words)
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [13]:
# Raw= True makes the code much faster
train['onegram_pct'] = train.apply(one_gram, axis=1, raw=True)
train['twogram_pct'] = train.apply(two_gram, axis=1, raw=True)
train['threegram_pct'] = train.apply(three_gram, axis=1, raw=True)

In [14]:
train['Cosine'] = train.apply(get_cosine, axis=1)

In [15]:
# Applying the function to the train dataset
train['match_share'] = train.apply(word_match_share, axis=1, raw=True)

## Working on the test data

In [16]:
#Applying all the functions to the test data.
test['qid1_count'] = test["question1"].apply(lambda x : len(str(x).split()))
test['qid2_count'] = test["question2"].apply(lambda x : len(str(x).split()))
#What is the maximum difference between the length of two 'marked' duplicate questions?
test['count12_diff']= test['qid1_count']-test['qid2_count']  
test['count12_diff'] = test['count12_diff'].abs()
test['match_share'] = test.apply(word_match_share, axis=1, raw=True)
test['onegram_pct'] = test.apply(one_gram, axis=1, raw=True)
test['twogram_pct'] = test.apply(two_gram, axis=1, raw=True)
test['threegram_pct'] = test.apply(three_gram, axis=1, raw=True)
test['Cosine'] = test.apply(get_cosine, axis=1)

## Xgboost

In [17]:
#Importing the library
import xgboost as xgb

D:\DS Tools\Anaconda2\envs\py35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [36]:
feature_list = ['qid1_count', 'qid2_count', 'count12_diff','match_share',
       'onegram_pct', 'twogram_pct', 'Cosine']

#Creating the predictors (X) and the Target Variables(Y) for the test and train data.
train_X = train[feature_list]
train_Y = train [['is_duplicate']]

train_y = xgb.DMatrix(train_Y)
train_x = xgb.DMatrix(train_X , label=train_Y)


In [37]:
#Prepping the test data.
test_X = test[feature_list]
test_x = xgb.DMatrix(test_X)

In [38]:
#Parameters of the model
param = {'max_depth':8, 'eta':0.02, 'silent':1, 'objective':'binary:logistic', 'eval_metric':'logloss' }
watchlist = [(train_x, 'train'), (test_x, 'test')]

In [39]:
#Training the model
model =  xgb.train(params= param , dtrain = train_x ,num_boost_round=400, verbose_eval=10   )

In [40]:
#Calculating the log-loss, cross entrophy on the train data.
from sklearn.metrics import log_loss
log_loss(train_Y,model.predict(train_x))

0.43875568813036891